In [13]:
import xgboost as xgb
print(xgb.__version__)

3.1.2


In [14]:
import sys, xgboost as xgb
print(sys.executable)        # should point to ...\.venv\Scripts\python
print(xgb.__version__)      
print(xgb.__file__)          # should live under ...\.venv\...

e:\Regression_E2E\.venv\Scripts\python.exe
3.1.2
e:\Regression_E2E\.venv\Lib\site-packages\xgboost\__init__.py


In [15]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

In [16]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv(r"E:\Regression_E2E\data\processed\feature_engineered_train.csv")
eval_df  = pd.read_csv(r"E:\Regression_E2E\data\processed\feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [17]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [22]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("file:///E:/Regression_E2E/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/12/15 13:28:43 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-15 13:28:43,388] A new study created in memory with name: no-name-77ad0588-b9b2-4520-853c-091c67e406e9
[I 2025-12-15 13:28:57,444] Trial 0 finished with value: 87877.03757364972 and parameters: {'n_estimators': 491, 'max_depth': 4, 'learning_rate': 0.010800669498175888, 'subsample': 0.7745216066270331, 'colsample_bytree': 0.5537823514818255, 'min_child_weight': 8, 'gamma': 0.9531087890559992, 'reg_alpha': 4.694461350001077, 'reg_lambda': 0.0001652049784400601}. Best is trial 0 with value: 87877.03757364972.
[I 2025-12-15 13:29:22,564] Trial 1 finished with value: 82327.06043018565 and parameters: {'n_estimators': 685, 'max_depth': 6, 'learning_rate': 0.26627454878714657, 'subsample': 0.7551674472674508, 'colsample_bytree': 0.8101516036759975, 'min_child_weight': 7, 'gamma': 0.7717791686904379, 'reg_alpha': 0.20296620991858888, 'reg_lambda'

Best params: {'n_estimators': 579, 'max_depth': 10, 'learning_rate': 0.021461195821350598, 'subsample': 0.8524009877024534, 'colsample_bytree': 0.6807424887545745, 'min_child_weight': 4, 'gamma': 3.1410783583712276, 'reg_alpha': 2.0398562954115748e-05, 'reg_lambda': 4.379917343082967}


In [23]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 30788.858133629994
RMSE: 69837.65051142688
R²: 0.9623088601197523


2025/12/15 13:46:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
